In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import insee
from datetime import datetime

ModuleNotFoundError: No module named 'insee'

# Merge raw data at insee tile level

In [ ]:
path = '/Users/anmusso/Desktop/PhD/Projects/Current/NetMob/NetMobData/data/FigureData/Sleep'
var_sleep = 'deviation_from_mean'

In [ ]:
mobile_traffic_data = pd.read_csv(f'{path}/deviation_from_mean.csv')
noise_data = pd.read_csv(f'{path}/noise_estimates_paris_n.csv')
admin_data = pd.read_csv(f'{path}/admin_data_paris.csv')
matching_tile_mobile_traffic_tile_insee = pd.read_csv(f'{path}/matching_tile_mobile_traffic_tile_insee_paris.csv')
full_data = pd.merge(mobile_traffic_data, noise_data, on='tile')
full_data = pd.merge(full_data, matching_tile_mobile_traffic_tile_insee, left_on='tile', right_on='tile_mobile_traffic')
full_data.drop(columns=['tile'], inplace=True)
full_data = pd.merge(full_data, admin_data, left_on='tile_insee', right_on='tile')
full_data.drop(columns=['tile'], inplace=True)

In [ ]:
def aggregation_function(x):
    area = full_data.loc[x.index, 'intersection_area'].values
    decibels = x.values
    decibels_to_linear_scale = np.power(10, decibels / 10)
    weighted_average_decibels_linear_scale = np.average(decibels_to_linear_scale, weights=area)
    average_decibels = 10 * np.log10(weighted_average_decibels_linear_scale)
    rounded_average_decibels = np.round(average_decibels, decimals=0)
    return rounded_average_decibels


def weighted_mean(x):
    area = full_data.loc[x.index, 'intersection_area'].values
    return np.average(x.values, weights=area)


aggregated_full_data = full_data.groupby('tile_insee').agg({'deviation_from_mean': weighted_mean, 'pop': 'first', 'income': 'first', 'night_equipment_counts': 'first', 'age': 'first', 'noise_estimate': aggregation_function})

In [ ]:
tile_insee_geometry = insee.tile.get_data(tile=list(aggregated_full_data.index.unique()), var_name='geometry')
aggregated_full_data = pd.merge(aggregated_full_data, tile_insee_geometry, left_index=True, right_index=True)
ts = pd.date_range(start=datetime(2019, 1, 1, 21), end=datetime(2019, 1, 2, 3, 30), freq='1min')
times = pd.date_range(start=datetime(2019, 1, 1, 21), end=datetime(2019, 1, 2, 3, 30), freq='15min')
multiplier = len(ts) / len(times)


def float_to_time(x):
    index = np.round(x * multiplier).astype(int)
    return ts[index].time()

In [ ]:
# aggregated_full_data['bed_time_index'] = aggregated_full_data['expected_time_number']
aggregated_full_data[('bed_time_index')] = -1*aggregated_full_data['deviation_from_mean']
aggregated_full_data.head()
aggregated_full_data.reset_index(names='tile_insee', inplace=True)
aggregated_full_data = gpd.GeoDataFrame(aggregated_full_data, geometry='geometry')
aggregated_full_data.to_crs(epsg=2154, inplace=True)
aggregated_full_data['centroid'] = aggregated_full_data.centroid

In [ ]:
aggregated_full_data.head()

# Matching 

In [11]:
def get_matching(df: pd.DataFrame, geo_var: str, treatment: str, high: float, low: float, buffer_size: int):
    high_var = df.loc[df[treatment] > high][[geo_var, 'centroid']]
    high_var = gpd.GeoDataFrame(high_var, geometry='centroid')
    high_var['buffer'] = high_var['centroid'].apply(lambda x: x.buffer(buffer_size))
    high_var.set_geometry('buffer', inplace=True)
    
    low_var = df.loc[df[treatment] < low][[geo_var, 'centroid']]
    low_var = gpd.GeoDataFrame(low_var, geometry='centroid')
    
    match_high_var_with_nearby_low_var_tiles = gpd.sjoin(high_var, low_var, predicate='intersects', lsuffix='high', rsuffix='low')
    matching_high_low_var = match_high_var_with_nearby_low_var_tiles[[f'{geo_var}_high', f'{geo_var}_low']]
    return matching_high_low_var

In [12]:
def get_metadata_matching(match, data, geo_var, treatment, outcome):
    vars_metadata = ['pop', 'income', 'night_equipment_counts', 'age', treatment, outcome]
    data_ = data.set_index(geo_var)
    data_high = data_.loc[match[f'{geo_var}_high'].values][vars_metadata].reset_index(names=f'{geo_var}')
    data_low = data_.loc[match[f'{geo_var}_low'].values][vars_metadata].reset_index(names=f'{geo_var}')
    data_high.rename(columns={c: f'{c}_high' for c in data_high.columns}, inplace=True)
    match_with_metadata = pd.concat([match.set_index(f'{geo_var}_high'), data_high.set_index(f'{geo_var}_high')], axis=1)
    match_with_metadata.reset_index(inplace=True)
    data_low.rename(columns={c: f'{c}_low' for c in data_low.columns}, inplace=True)
    match_with_metadata = pd.concat([match_with_metadata.set_index(f'{geo_var}_low'), data_low.set_index(f'{geo_var}_low')], axis=1)
    match_with_metadata.reset_index(inplace=True)
    return match_with_metadata
    

In [22]:
matching = get_matching(aggregated_full_data.copy(), geo_var='tile_insee', treatment='noise_estimate', high=60, low=48, buffer_size=350)

In [23]:
matching_with_metadata = get_metadata_matching(matching, aggregated_full_data.copy(), geo_var='tile_insee', treatment='noise_estimate', outcome='bed_time_index')

In [24]:
matching_with_metadata.to_csv(f'{path}/matching_figure.csv', index=False)

In [16]:
import webbrowser
def plot_map(df, column, legend=True, tiles="CartoDB positron"):
    path_map = '/Users/anmusso/Desktop/PhD/Projects/Current/NetMob/NetMobCode/temp/map_matching.html'
    html_map = df.explore(column=column, legend=legend, tiles=tiles)
    html_map.save(path_map)
    webbrowser.open(f'file://{path_map}')

In [30]:
a = aggregated_full_data.set_index('tile_insee')
matching_with_metadata = matching_with_metadata[matching_with_metadata['pop_high'] > 1000]
t1 = matching_with_metadata.iloc[0:10]['tile_insee_high'].values
t2 = matching_with_metadata.iloc[0:10]['tile_insee_low'].values
t = np.append(t1, t2)
b = a.loc[t]

In [31]:
plot_map(b, column='noise_estimate')

# Regression

In [17]:
vars_regression = ['pop', 'income', 'night_equipment_counts', 'age', 'noise_estimate', 'bed_time_index']
aggregated_full_data[vars_regression].to_csv(f'{path}/regression_figure.csv', index=False)

# Map

In [18]:
map_vars = ['income','age', 'noise_estimate', 'bed_time_index', 'geometry']
aggregated_full_data[map_vars].to_file(f'{path}/map_figure.geojson', driver='GeoJSON')